In [ ]:
from bs4 import BeautifulSoup
import requests

url = 'https://www.eprbulletsafsc.com/'
page = requests.get(url)
soup = BeautifulSoup(page.content, 'html.parser')

In [ ]:
from collections import defaultdict


anchor_tags = soup.find_all('a')
required_urls = defaultdict(list)
for tag in anchor_tags:
    href = tag['href']
    category = tag.text.split(' ')[0]

    cat_prefix = category[0] + 'XXXX'

    linked_page_url = url + "/" + href

    if not category[0].isdigit(): 
        required_urls['General'].append(linked_page_url)
    else:
        required_urls[cat_prefix].append(linked_page_url)

print(required_urls)

In [ ]:
res = {}
for category in required_urls:
    linked_page_urls = required_urls[category]
    if category in res: continue # Prevent Overwriting

    res[category] = []

    for linked_page_url in linked_page_urls:
        linked_page = requests.get(linked_page_url)
        linked_soup = BeautifulSoup(linked_page.content, 'html.parser')
        divSoup = linked_soup.find("div", {"class" : "main"})
        ps = divSoup.findAll('p')

        for p in ps:
            splits = p.text.split('\n')

            for line in splits:
                if line != '' and line[2:] not in res[category]: # Not having a check somehow leads to duplicates
                    res[category].append(line[2:])
                
                if len(res[category]) == 20: break

            if len(res[category]) == 20: break

        if len(res[category]) == 20: break

print(res)

In [ ]:
import pandas as pd

for key in res:
    print(len(res[key]))

df = pd.DataFrame(res.items(), columns=['Category', 'Data']).explode('Data').reset_index(drop=True)
print(df)
df.to_excel('output.xlsx', index=False)